# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [5]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [7]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [14]:
data.shape

(4870, 2)

In [15]:
# number of callbacks for black-sounding names
white_calls = sum(data[data.race=='w'].call)
black_calls = sum(data[data.race=='b'].call)
print("White Calls:", white_calls)
print("Black Calls:", black_calls)

White Calls: 235.0
Black Calls: 157.0


In [16]:

w = data[data.race=='w']
b = data[data.race=='b']

print("Rows of white names:", w.shape[0])
print("Rows of black names:", b.shape[0])

Rows of white names: 2435
Rows of black names: 2435


In [17]:
pw = 235/2435
pb = 157/2435
print("n(pw) =", pw*4870)
print("n(pb) =", pb*4870)
print("n(1-pw) =", (1-pw)*4870)
print("n(1-pb) =", (1-pb)*4870)

n(pw) = 470.0
n(pb) = 314.0
n(1-pw) = 4400.0
n(1-pb) = 4556.0


 #### 1. What test is appropriate for this problem? Does CLT apply?


The appropriate test for this problem is a hypothesis test comparing population proportions.
CLT applies because our sample size is large (n = 4870) and observations are independent.

 #### 2. What are the null and alternate hypotheses? 

Null Hypothesis: the mean difference between callback rate for black & white-sounding names is zero
Alternative Hypothesis: there is a significant (non-zero) difference between callback rate for black & white-sounding names

#### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

Frequentist Statistics

In [18]:
#Compute z-score for 99% Confidence Interval
import scipy.stats as stats
z_score = stats.norm.ppf(.995)

#Compute Margin of Error
import numpy as np
margin_error = z_score*np.sqrt(pw*(1-pw)/w.shape[0]+pb*(1-pb)/b.shape[0])
print("Margin of Error=", margin_error, ".")

Margin of Error= 0.02004863403754258 .


In [19]:
#Compute Difference of Proportions
p_diff = pw-pb

#Compute Confidence Interval
c_min = p_diff - margin_error
c_max = p_diff + margin_error

print('We are 99% Confident that (', c_min, ",", c_max, ") captures the true difference in proportions.")

We are 99% Confident that ( 0.011984220171903006 , 0.05208148824698816 ) captures the true difference in proportions.


In [20]:
z_score2 = p_diff/np.sqrt(pw*(1-pw)/w.shape[0]+pb*(1-pb)/b.shape[0])
p = 1-stats.norm.cdf(z_score2)
print("The probability that the difference in proportion is due to chance is", p, ".")

The probability that the difference in proportion is due to chance is 1.9312826037620745e-05 .


Bootstrapping Method

In [22]:
# Define difference function to compute difference in proportions
def diff_of_p(x,y):
    return abs(np.sum(x)-np.sum(y))


# Define bootstrap function that randomly splits data
def bs_split_data(data, func, size=1):
    
    # Initialize replicates
    bs_replicates = np.empty(size)
    
    for i in range(size):
        # Permute the concatenated array: permuted_data
        permuted_data = np.random.permutation(data)

        # Split the permuted array into two: perm_sample_1, perm_sample_2
        # Assume data can be split into even numbers
        half_count = int(len(data)/2)
        perm_sample_1 = permuted_data[:half_count]
        perm_sample_2 = permuted_data[half_count:]
    
        # Call function
        bs_replicates[i] = func(perm_sample_1/half_count, perm_sample_2/half_count)
        
    return bs_replicates

# Acquire 10000 bootstrap replicates using difference of proprotion
bs_replicates = bs_split_data(data.call, diff_of_p, size=10000)

results = np.sum(bs_replicates>=diff_of_p(pw,pb))/len(bs_replicates)

print("p-value =", results)

p-value = 0.0001


In [26]:
# Acquire 100000 bootstrap replicates using difference of proprotion
bs_replicates_6 = bs_split_data(data.call, diff_of_p, size=100000)

results = np.sum(bs_replicates_6>=diff_of_p(pw,pb))/len(bs_replicates_6)

print("p-value =", results)

p-value = 3e-05


It's a smaller percentage and closer to the p-value obtained using the frequentist method above.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

#### 4.Write a story describing the statistical significance in the context or the original problem.

With a p-value of less than 0.05 and confidence interval that does not contain 0, we can conclude that there is a statistically significant difference between callback rates for black and white-sounding names.

####  5.Does the analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, the conclusion above does not imply that race is the most important factor in callback success, because it solely examines race. All other relevant factors like education and years of experience would need to be examined and compared in order to identify the *most important* one.